In [106]:
# General libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
# For preprocessing
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
from skimage.color import rgb2lab, lab2rgb, rgb2gray
# For NN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, UpSampling2D
from tensorflow.keras.callbacks import EarlyStopping
# For output image
from skimage.io import imsave

%matplotlib inline

In [97]:
# Create function that preprocesses images
input_image = load_img(path = "Images/Train/0A9kTN.jpg")
input_array = img_to_array(input_image)
# input_array = resize(image = input_array, output_shape = (28, 28, 3))
input_array = np.array(input_array, dtype = float)

imsave("Outputs/img_input.png", input_array)

print("input_array shape:", input_array.shape, type(input_array))

Lossy conversion from float64 to uint8. Range [16.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


input_array shape: (256, 256, 3) <class 'numpy.ndarray'>


In [99]:
# Create Lab image
X = rgb2lab(1.0/255 * input_array)[:, :, 0] # 1.0/255 indicates that we are using a 24-bit RGB color space. It means that we are using 0-255 numbers for each colour channel.
Y = rgb2lab(1.0/255 * input_array)[:, :, 1:]
Y /= 128
X = X.reshape(1, 256, 256, 1) # (batchsize, height, width, colourchannel)
Y = Y.reshape(1, 256, 256, 2)
print("X shape:", X.shape, type(X))
print("Y shape:", Y.shape, type(Y))

X shape: (1, 256, 256, 1) <class 'numpy.ndarray'>
Y shape: (1, 256, 256, 2) <class 'numpy.ndarray'>


In [127]:
# Building the neural network

model = Sequential(name = "Colouring-FashionMNIST")
# We must define the input shape, the model cannot know it
# None dimension means that the network will be able to accept inputs of any dimension. 
model.add(InputLayer(input_shape = (None, None, 1)))
model.add(Conv2D(name = "Conv2D_8a_3x3", filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(Conv2D(name = "Conv2D_8b_3x3", filters = 8, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_16a_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_16b_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(Conv2D(name = "Conv2D_32a_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(Conv2D(name = "Conv2D_32b_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same', strides = 2))
model.add(UpSampling2D(name = "UpSamp-a_2x2", size = (2, 2))) # The upsampling factors for rows and columns
model.add(Conv2D(name = "Conv2D_32c_3x3", filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D(name = "UpSamp-b_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_16c_3x3", filters = 16, kernel_size = (3, 3), activation = 'relu', padding = 'same'))
model.add(UpSampling2D(name = "UpSamp-c_2x2", size = (2, 2)))
model.add(Conv2D(name = "Conv2D_2_3x3", filters = 2, kernel_size = (3, 3), activation = 'tanh', padding = 'same'))


# Note about maintaining image ratio:
# Classification networks only care about the final classification. Therefore, the keep decreasing the image size and quality as it moves through the network.
# Colouring networks keep the image ratio. This is done by adding white padding. Otherwise, each convolutional layer cuts the image. It's done with the padding = 'same' parameter.

# Note about UpSampling:
# The double the size of the image, the colouring network uses an upsampling layer.

In [128]:
model.compile(optimizer = 'rmsprop', loss = 'mse')

print(model.summary())

Model: "Colouring-FashionMNIST"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv2D_8a_3x3 (Conv2D)       (None, None, None, 8)     80        
_________________________________________________________________
Conv2D_8b_3x3 (Conv2D)       (None, None, None, 8)     584       
_________________________________________________________________
Conv2D_16a_3x3 (Conv2D)      (None, None, None, 16)    1168      
_________________________________________________________________
Conv2D_16b_3x3 (Conv2D)      (None, None, None, 16)    2320      
_________________________________________________________________
Conv2D_32a_3x3 (Conv2D)      (None, None, None, 32)    4640      
_________________________________________________________________
Conv2D_32b_3x3 (Conv2D)      (None, None, None, 32)    9248      
_________________________________________________________________
UpSamp-a_2x2 (UpSampling2D)  (None, None, No

In [129]:
# Train the model
es = EarlyStopping(monitor = 'loss', 
                   mode = 'min', 
                   patience = 50, 
                   verbose = 1, # verbose = {0, 1, 2}
                   restore_best_weights = True)

model.fit(x = X,
          y = Y, 
          batch_size = 1, 
          epochs = 1000, 
          callbacks = [es], 
          verbose = 1)


Epoch 1/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.1677
Epoch 2/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.1703
Epoch 3/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.1013
Epoch 4/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0357
Epoch 5/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0340
Epoch 6/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0320
Epoch 7/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0297
Epoch 8/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0267
Epoch 9/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0228
Epoch 10/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0283
Epoch 11/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0280
Epoch 12/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0266
Epoch 13/1000
1/1 [==============================

1/1 [==============================] - 0s 1ms/step - loss: 0.0143
Epoch 104/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0125
Epoch 105/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0110
Epoch 106/1000
1/1 [==============================] - 0s 3ms/step - loss: 0.0105
Epoch 107/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0102
Epoch 108/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0116
Epoch 109/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0106
Epoch 110/1000
1/1 [==============================] - 0s 4ms/step - loss: 0.0099
Epoch 111/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0127
Epoch 112/1000
1/1 [==============================] - 0s 981us/step - loss: 0.0178
Epoch 113/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0150
Epoch 114/1000
1/1 [==============================] - 0s 2ms/step - loss: 0.0129
Epoch 115/1000
1/1 [=====================

1/1 [==============================] - 0s 1ms/step - loss: 0.0141
Epoch 205/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0137
Epoch 206/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0091
Epoch 207/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0078
Epoch 208/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0066
Epoch 209/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0062
Epoch 210/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0114
Epoch 211/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0108
Epoch 212/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0108
Epoch 213/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0086
Epoch 214/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0071
Epoch 215/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0065
Epoch 216/1000
1/1 [=======================

1/1 [==============================] - 0s 1ms/step - loss: 0.0089
Epoch 306/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0208
Epoch 307/1000
1/1 [==============================] - 0s 1000us/step - loss: 0.0145
Epoch 308/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0101
Epoch 309/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0076
Epoch 310/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0063
Epoch 311/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0044
Epoch 312/1000
1/1 [==============================] - 0s 990us/step - loss: 0.0034
Epoch 313/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0081
Epoch 314/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0068
Epoch 315/1000
1/1 [==============================] - 0s 1000us/step - loss: 0.0067
Epoch 316/1000
1/1 [==============================] - 0s 973us/step - loss: 0.0050
Epoch 317/1000
1/1 [=============

In [130]:
print(model.evaluate(X, Y, batch_size = 1))

1/1 [==============================] - 0s 861us/step - loss: 0.0035
0.0035336429718881845


In [131]:
output = model.predict(X)
output *= 128

print("Output shape:", output.shape, type(output))

# Output colourisation
output_array = np.zeros((256, 256, 3))
output_array[:, :, 0] = X[:, :, 0]
output_array[:, :, 1:] = output[0]
print("output_array shape:", output_array.shape, type(output))



imsave("Outputs/img_result.png", lab2rgb(output_array))
imsave("Outputs/img_result_gray.png", rgb2gray(lab2rgb(output_array)))

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


Output shape: (1, 256, 256, 2) <class 'numpy.ndarray'>
output_array shape: (256, 256, 3) <class 'numpy.ndarray'>


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
